In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
# pd.set_option('display.max_rows', 50)
import tabula
from pandasql import sqldf
import math
from scipy.stats import chi2
from scipy import stats
from matplotlib import pyplot as plt

In [2]:
# DNA_inside_w = pd.read_excel(r"C:\Users\edwar\Desktop\農藝\paper2\data collection\integrated_results_DNAinside.xlsx", sheet_name = 'weighted')
RNA_inside_w = pd.read_excel(r"C:\Users\edwar\Desktop\農藝\paper2\data collection\integrated_results_RNAinside.xlsx", sheet_name = 'weighted')

In [ ]:
RNA_data = RNA_inside_w[['gene', 'weight_score']].rename(columns = {'weight_score':'score'})
df = RNA_data.copy()
gene_data = pd.DataFrame()
gene_data['gene'] = RNA_inside_w['gene']
gene_data['times'] = 0

bootstrap_times = 1000
permute_times = 10
n =  150 # number of prioritized genes

for bootstrap_time in range(bootstrap_times):
    # bootstrapping
    bootstrap_data = RNA_data.sample(n = len(RNA_data['gene']), replace = True).sort_values(by = ['score'], ascending = False).reset_index().drop(['index'], axis = 1)[:n]
    
    for permute_time in range(permute_times):
        # permutation
        np.random.shuffle(df['score'].values)
        df_v1 = df[df['gene'].isin(list(bootstrap_data['gene']))].reset_index().drop(['index'], axis = 1)
        # records of times of bootstrap score over permuted score
        bootstrap_data_v1 = bootstrap_data.merge(df_v1.rename(columns = {'score':'score_permute'}), on = ['gene'], how = 'left')
        bootstrap_data_v1['diff'] = bootstrap_data_v1['score'] - bootstrap_data_v1['score_permute']
        bootstrap_data_v1['times'] = 0
        bootstrap_data_v1.loc[(bootstrap_data_v1['diff'] > 0), 'times'] = 1
        # bootstrap_data_v1 = bootstrap_data_v1.drop(['score_permute'], axis = 1)
        
        # sum up times of bootstrap score over permuted score
        add_data = bootstrap_data_v1[bootstrap_data_v1['times'] == 1].reset_index().drop(['index'], axis = 1)[['gene', 'times']]
        add_data_v1 = pd.DataFrame()
        gene_list = list(add_data['gene'].unique())
        for gene in gene_list:
            df_gene = add_data[add_data['gene'] == gene].reset_index().drop(['index'], axis = 1)
            if len(df_gene['gene']) > 1:
                df_gene_v1 = pd.DataFrame()
                df_gene_v1['gene'] = df_gene['gene'].unique()
                df_gene_v1['times'] = df_gene['times'].sum()
                add_data_v1 = pd.concat([add_data_v1, df_gene_v1]).reset_index().drop(['index'], axis = 1)

            elif len(df_gene['gene']) == 1:
                add_data_v1 = pd.concat([add_data_v1, df_gene]).reset_index().drop(['index'], axis = 1)
        
        # totally sum up
        gene_data = gene_data.merge(add_data_v1.rename(columns = {'times':'times_delta'}), on = ['gene'], how = 'left')
        gene_data['times'] = gene_data[['times', 'times_delta']].sum(axis = 1)
        gene_data = gene_data.drop(['times_delta'], axis = 1)
    
    if (bootstrap_time + 1) % 100 == 0:
        print(bootstrap_time + 1)